### Official Websites (Examples)
https://bashtage.github.io/linearmodels/panel/index.html

### Official Github (Examples)

https://github.com/bashtage/linearmodels/blob/main/README.md

https://bashtage.github.io/linearmodels/panel/examples/examples.html

## 1. Wage (Wage ~ Experience)

In [1]:
from linearmodels.panel import PanelOLS
from linearmodels.datasets import wage_panel
import statsmodels.api as sm
from linearmodels import BetweenOLS, FirstDifferenceOLS, PooledOLS
import pandas as pd

pd.set_option('display.float_format', lambda x:'%.7f' % x)

In [2]:
import stata_setup
stata_setup.config("/Applications/Stata/", "se")


[0.014s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016d3f4000-0x000000016d400000).
[0.014s][warning][os,thread] Attempt to deallocate stack guard pages failed.

  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: 100-student lab perpetual
Serial number: 401706316154
  Licensed to: 52mac
               www.52mac.com

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000; see help set_maxvar.


In [16]:
'''
%%stata
sysuse auto, clear
summarize mpg
scatter mpg price
'''


'\n%%stata\nsysuse auto, clear\nsummarize mpg\nscatter mpg price\n'

In [23]:
import pandas as pd
import io
import requests

wagedata = pd.read_csv('wage_panel.csv')
wagedata


,Unnamed: 0,nr,year,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation
0,0,13,1980,0,1,0,2672,0,14,0,1.1975400,1,9
1,1,13,1981,0,2,0,2320,0,14,1,1.8530600,4,9
2,2,13,1982,0,3,0,2940,0,14,0,1.3444620,9,9
3,3,13,1983,0,4,0,2960,0,14,0,1.4332130,16,9
4,4,13,1984,0,5,0,3071,0,14,0,1.5681250,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4355,4355,12548,1983,0,8,0,2080,1,9,0,1.5918790,64,5
4356,4356,12548,1984,0,9,0,2080,1,9,1,1.2125430,81,5
4357,4357,12548,1985,0,10,0,2080,1,9,0,1.7659620,100,5
4358,4358,12548,1986,0,11,0,2080,1,9,1,1.7458940,121,5


In [27]:
%%stata
sysuse auto, clear
summarize wagedata

Exception in thread Stata:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 176, in run
    raise SystemError(output)
SystemError: 
. sysuse auto, clear
(1978 automobile data)

. summarize wagedata
variable wagedata not found
r(111);
r(111);



In [28]:
%%stata -d wagedata -eret myeret
xtset nr year
xtreg lwage expersq married union i.year, fe vce(cluster nr)
ereturn list



. xtset nr year

Panel variable: nr (strongly balanced)
 Time variable: year, 1980 to 1987
         Delta: 1 unit

. xtreg lwage expersq married union i.year, fe vce(cluster nr)

Fixed-effects (within) regression               Number of obs     =      4,360
Group variable: nr                              Number of groups  =        545

R-squared:                                      Obs per group:
     Within  = 0.1806                                         min =          8
     Between = 0.0286                                         avg =        8.0
     Overall = 0.0888                                         max =          8

                                                F(10,544)         =      46.59
corr(u_i, Xb) = -0.1222                         Prob > F          =     0.0000

                                   (Std. err. adjusted for 545 clusters in nr)
------------------------------------------------------------------------------
             |               Robust
       

————————————————————————————————————————————————————————————————————————————————————————————————————————————

In [ ]:
data = wage_panel.load()
data

In [ ]:
# data.to_csv("wage_panel.csv")

In [ ]:
# Convert the year into categorical type
year = pd.Categorical(data.year)

data = data.set_index(['nr','year'])

In [ ]:
data["year"] = year
data.info()

In [ ]:
print(wage_panel.DESCR)
print(data.head())

### 1.1 Descriptive Statistics

In [ ]:
data.describe()

In [ ]:
print(data.columns)

In [ ]:
# Draw the Correlation heatmap
import seaborn as sns
import matplotlib.pyplot as plt

# set the font size of the heatmap
sns.set(font_scale=0.6)
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.show()

### 1.2 Basic Regression (Pooled OLS) on Panel Data

- PooledOLS is just plain OLS that understands that various panel data structures. It is useful as a base model

$$
y_{it} = \beta x_{it} + (\alpha + \epsilon_{it})
$$

In [ ]:
# Determine the exogenous variables
exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog = sm.add_constant(data[exog_vars])
model = PooledOLS(data.lwage, exog)
pooled_res = model.fit()
print(pooled_res)

### 1.3 Entity Effect

When modeling panel data it is common to consider models beyond what OLS will efficiently estimate. The most common are error component models which add an additional term to the standard OLS model,

$$
y_{it} = \beta x_{it} + \alpha_i + \epsilon_{it}
$$

where $\alpha_i$ affects all values of entity i. 

When the $\alpha_i$ are uncorrelated with the regressors in $x_{it}$ , 

a random effects model can be used to efficiently estimate parameters of this model.

#### 1.3.1 Random Effect

The random effects model is <font color = "red"> virtually identical to the pooled OLS model </font>  <font color = "orange">except that is accounts for the structure of the model and so is more efficient </font>. Random effects uses a quasi-demeaning strategy which subtracts the time average of the within entity values to account for the common shock.

In [ ]:
from linearmodels import RandomEffects

mod = RandomEffects(data.lwage, exog)
re_res = mod.fit()
print(re_res)

- The model fit is fairly similar, although the return to experience has changed substantially, as has its significance. 

- <font color = "orange"> This is partially explainable by the inclusion of the year dummies </font> which will fit the trend in experience and so only the cross-sectional differences matter. 

- The quasi-differencing in the random effects estimator depends on a quantity that depends on the relative variance of the idiosyncratic shock and the common shock. 

- This can be accessed using variance_decomposition.

In [ ]:
print(re_res.variance_decomposition)
re_res.theta.head()

The coefficient $\theta_i$ determines how much demeaning takes place. When this value is 1, the RE model reduces to the pooled model since this occurs when there is no variance in the effects. When panels are unbalanced it will vary across entities, but in this balanced panel all values are the same.

#### 1.3.2 Between Estimator 

The between estimator is an alternative, usually less efficient estimator, can can be used to estimate model parameters. 

It is particular simple since it first computes the time averages of y and x and then runs a simple regression using these averages.

The year dummies are dropped since the averaging removes differences due to the year. expersq was also dropped since it is fairly co-linear with exper. These results are broadly similar to the previous models.

In [ ]:
exog_vars = ["black","hisp","exper","married","educ","union"]
exog = sm.add_constant(data[exog_vars])
mod = BetweenOLS(data.lwage, exog)
be_res = mod.fit()
print(be_res)

### 1.3.3 Entity effect
Entity effects are included by setting entity_effects=True. This is equivalent to including dummies for each entity. In this panel, this would add 545 dummy variables and estimation of the model would be considerably slower. PanelOLS does not actually use dummy variables and instead uses group-wise demeaning to achieve the same effec

In [ ]:
exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

### 1.3.4 Time Effect

Time effect can be added using time_effects=True. Here the time dummies are removed. Note that the core coefficients are identical. The only change is in the test statistic for poolability since not the “effects” include both entity and time, whereas before only entity were included.

In [ ]:
exog_vars = ["expersq", "union", "married"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, time_effects=True)
fe_te_res = mod.fit()
print(fe_te_res)

## 1.4 First Differences

First differencing is an alternative to using fixed effects when there might be correlation. When using first differences, time-invariant variables must be excluded. Additionally, only one linear time-trending variable can be included since this will look like a constant. This variable will soak up all time-trends in the data, and so interpretations of these variable can be challenging.

In [ ]:
from linearmodels.panel import FirstDifferenceOLS

exog_vars = ["exper", "expersq", "union", "married"]
exog = data[exog_vars]
mod = FirstDifferenceOLS(data.lwage, exog)
fd_res = mod.fit()
print(fd_res)

## 1.5 Comparing Models

Model results can be compared using *compare*. compare accepts lists of results, a dictionary of results where the key is interpreted as the model name.

In [ ]:
from linearmodels.panel import compare

print(compare({"BE": be_res, "RE": re_res, "Pooled": pooled_res}))

## 1.6 Covariance Options

### 1.6.1 Heteroskedasticity Robust Covariance
White”s robust covariance can be used by setting cov_type="robust. This estimator adds some robustness against certain types of specification issues but should not be used when using fixed effects (entity effects) since it is no longer robust. Instead a clustered covariance is required.

In [ ]:
exog_vars = ["expersq", "married", "union"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, time_effects=True)
robust = mod.fit(cov_type="robust")
none = mod.fit()
print(robust)
print(compare({"Roubst": robust, "None": none}))


## 1.6.2 Clustered by Entity

The usual variable to cluster are are entity or entity and time. The can be implemented using cov_type="clustered" and the additional keyword arguments cluster_entity=True and/or cluster_time=True.

In [ ]:
clust_entity = mod.fit(cov_type="clustered", cluster_entity=True)
print(clust_entity)

In [ ]:
clust_entity_time = mod.fit(
    cov_type="clustered", cluster_entity=True, cluster_time=True
)
print(clust_entity_time)

## 1.6.3 Use "OrderedDict" to compare the models

In [ ]:
from collections import OrderedDict

res = OrderedDict()
res["Robust"] = robust
res["Entity"] = clust_entity
res["Entity - Time"] = clust_entity_time
print(compare(res))

## 1.6.4 Other Clusters 
Other clusters can be used by directly passing integer arrays (1 or 2 columns, or a 1-d array) using the input clusters. This example clustered by occupation, which is probably not a reliable variable to cluster on since there are only 9 groups and the usual theory for clustered standard errors requires that the number of clusters is large.

In [ ]:
clust_entity = mod.fit(cov_type="clustered", clusters=data.occupation)
print(data.occupation.value_counts())
print(clust_entity)

## 2. Grunfeld data 
using formulas to specify models

    invest  - Gross investment in 1947 dollars
    value   - Market value as of Dec. 31 in 1947 dollars
    capital - Stock of plant and equipment in 1947 dollars
    firm    - General Motors, US Steel, General Electric, Chrysler,
            Atlantic Refining, IBM, Union Oil, Westinghouse, Goodyear,
            Diamond Match, American Steel
    year    - 1935 - 1954

In [ ]:
# Load data
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data.to_csv('grunfeld.csv')

In [ ]:
# reset the index
data = data.set_index(['firm','year'])

### PanelOLS with Entity Effects¶

Entity effects are specified using the special command EntityEffects. By default a constant is not included, and so if a constant is desired, 1+ should be included in the formula. When including effects, the model and fit are identical whether a constant is included or not.

In [ ]:
### No Constant
mod = PanelOLS.from_formula("invest ~ value + capital + EntityEffects", data = data)
print(mod.fit())

In [ ]:
### Add Constant
mod = PanelOLS.from_formula("invest ~ 1 + value + capital + EntityEffects", data = data)
print(mod.fit())

### PanelOLS with Entity Effects & Time Effects

Time effects can be similarly included using *TimeEffects*. In many models, time effects can be consistently estimated and so they could be equivalently included in the set of regressors using a categorical variable.

In [ ]:
mod = PanelOLS.from_formula(
    "invest ~ 1 + value + capital + EntityEffects + TimeEffects", data = data
)
print(mod.fit())

### Between OLS

In [ ]:
mod = BetweenOLS.from_formula("invest ~ 1 + value + capital", data=data)
print(mod.fit())

### First Difference OLS

In [ ]:
mod = FirstDifferenceOLS.from_formula("invest ~ value + capital", data=data)
print(mod.fit())

### Pooled OLS

The pooled OLS estimator is a special case of PanelOLS when there are no effects. It is effectively identical to OLS in statsmodels (or WLS) but is included for completeness.

In [ ]:
mod = PooledOLS.from_formula("invest ~ 1 + value + capital", data = data)
print(mod.fit())